# Gerenciador de investimentos

Bibliotecas e opções

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import lib as l

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2'])

## Leitura dos dados

In [3]:
df = pd.read_csv('br_invest.csv')
df['Data'] = df['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())

In [18]:
dport = pd.read_csv('br_portfolio.csv')
dport['Date'] = dport['Date'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dprices = pd.read_csv('br_prices.csv')
di = l.generate_invest_table(dport, dprices)
dinvest = pd.concat([df, di]).reset_index(drop=True)

## Fluxos de investimentos

### Histórico de fluxos

In [19]:
dflow = dinvest.groupby('ID').apply(l.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow = dflow.sort_values(by=['Rendimento anual'], ascending=False).reset_index(drop=True)
dflow

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,94.100,2.000,7.19%,3.53%,51.54%,722.670,True,IVVB11
1,5.100,1059.610,2.300,0.492,0.22%,0.44%,5.42%,57.532,True,Nu
2,1.000,12491.070,48.940,1.115,0.39%,0.35%,4.29%,537.618,False,Nu
3,3.100,1989.970,9.020,1.311,0.45%,0.35%,4.21%,84.220,True,Nu
4,2.000,715.200,2.270,0.951,0.32%,0.33%,4.07%,29.194,False,Nu
5,4.100,10.100,0.020,0.623,0.2%,0.32%,3.87%,0.392,False,Nu
6,1.100,10940.620,57.890,1.770,0.53%,0.3%,3.63%,399.408,True,Nu
7,3.000,2002.290,1.620,0.459,0.08%,0.18%,2.13%,42.701,False,Nu
8,102.000,1390.300,11.800,0.590,0.85%,1.44%,18.69%,262.109,True,IVVB11
9,101.000,875.000,85.500,1.508,9.77%,6.38%,109.54%,1052.164,True,BRAX11


### Fluxos ativos

In [20]:
dflow[dflow['Ativo']]

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,94.100,2.000,7.19%,3.53%,51.54%,722.670,True,IVVB11
1,5.100,1059.610,2.300,0.492,0.22%,0.44%,5.42%,57.532,True,Nu
3,3.100,1989.970,9.020,1.311,0.45%,0.35%,4.21%,84.220,True,Nu
6,1.100,10940.620,57.890,1.770,0.53%,0.3%,3.63%,399.408,True,Nu
8,102.000,1390.300,11.800,0.590,0.85%,1.44%,18.69%,262.109,True,IVVB11
9,101.000,875.000,85.500,1.508,9.77%,6.38%,109.54%,1052.164,True,BRAX11
11,6.000,846.360,0.290,0.361,0.03%,0.1%,1.14%,9.679,True,Nu


## Performance

In [21]:
dnu = dflow[dflow['Meses'] > 0]
dnu = dnu[dnu['Tipo'] == 'Nu']

detf = dflow[dflow['Meses'] > 0]
detf = detf[detf['Tipo'] != 'Nu']

### Renda fixa

In [23]:
gain = dnu['Ganho'].sum()
value = dnu[dnu['Ativo']]['Valor inicial'].sum() + dnu[dnu['Ativo']]['Ganho'].sum()
total_yield = gain / dnu['Valor inicial'].sum()
year_profit = dnu[dnu['Ativo']]['Ganho em 1 ano'].sum()
annual_yield = year_profit / value
dav = pd.DataFrame([[value, gain, l.int2pct(total_yield), year_profit, l.int2pct(annual_yield)]], 
                   columns=['Valor atual', 'Ganho atual', 'Rendimento atual', 'Ganho em 1 ano', 'Rendimento anual'])
dav = l.change_df_prop(dav, 28)
dav

Valor atual,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
14906.1,122.72,0.4%,550.839,3.7%


### Renda Variável

In [14]:
gain = detf['Ganho'].sum()
value = detf[detf['Ativo']]['Valor inicial'].sum() + gain
total_yield = gain / detf['Valor inicial'].sum()
year_profit = detf[detf['Ativo']]['Ganho em 1 ano'].sum()
annual_yield = year_profit / value
dav = pd.DataFrame([[value, gain, l.int2pct(total_yield), year_profit, l.int2pct(annual_yield)]], 
                   columns=['Valor atual', 'Ganho atual', 'Rendimento atual', 'Ganho em 1 ano', 'Rendimento anual'])
dav = l.change_df_prop(dav, 28)
dav

Valor atual,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
3764.7,191.4,5.36%,2036.94,54.11%


### Mensal

In [15]:
dret = l.group_sum(l.calc_flow_gains(dinvest), group=['Tipo'], date='month').fillna(0)
dret['Total'] = dret.sum(axis=1)
cols = dret.columns
for c in cols:
    dret = l.window_mean(dret, c)

In [16]:
fig = go.FigureWidget()
palette1 = {'Nu': lcolors[3], 'BRAX11': lcolors[2], 'IVVB11': lcolors[0]}
palette2 = {'Nu': dcolors[3], 'BRAX11': dcolors[2], 'IVVB11': dcolors[0]}
for c in cols:
    if c == 'Total':
        continue
    fig = l.plot_bar(dret, fig=fig,  y=c, color=palette1[c], name='Retorno (' + c + ')')
    fig = l.plot_line(dret, fig=fig, y=(c+'Avg'), color=palette2[c], name='Retorno médio (' + c + ')')
fig

FigureWidget({
    'data': [{'marker': {'color': '#FFB84C'},
              'name': 'Retorno (BRAX11)',
       …

In [17]:
fig = l.plot_bar(dret, y='Total', color=lcolors[0], name='Retorno')
l.plot_line(dret, fig=fig, y='TotalAvg', color=dcolors[0], name='Retorno médio')

FigureWidget({
    'data': [{'marker': {'color': '#13E881'},
              'name': 'Retorno',
              't…